<a href="https://colab.research.google.com/github/rhysdavies21/library/blob/master/book_sabr_%26_sabr_lmm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##SABR and SABR LIBOR Market Models in Practice --- Crispoldi, Wigger & Larkin##

In [44]:
# Black formula
# Reference SABR & SABR LIBOR Market Models, page 32, table 4.1
# Note that this is NOT the discounted price

from scipy.stats import norm
import math

def Black(F_0, y, expiry, vol, isCall):
  '''
  Compute the Black formula.
  @var F_0: Forward rate at time 0
  @var y: option strike
  @var expiry: option expiry (in years)
  @var vol: Black implied volatility
  @var isCall: True or False
  '''
  option_value = 0
  if expiry * vol == 0.0:
    if isCall: 
      option_value = max(F_0 - y, 0.0)
    else:
      option_value = max(y - F_0, 0.0)
  else:
    d1 = dPlusBlack(F_0 = F_0, y = y, expiry = expiry, vol = vol)
    d2 = dMinusBlack(F_0 = F_0, y = y, expiry = expiry, vol = vol)
    if isCall:
      option_value = (F_0 * norm.cdf(d1) - y * norm.cdf(d2))
    else:
      option_value = (y * norm.cdf(-d2) - F_0 * norm.cdf(-d1))
  return option_value


def dPlusBlack(F_0, y, expiry, vol):
  '''
  Compute the d+ term appearing in the Black formula
  @var F_0: forward rate at time 0
  @vary y: option strike
  @var expiry: option expiry (in years)
  @var vol: Black implied volatility
  '''
  d_plus =  ((math.log(F_0 / y) + 0.5 * vol * vol * expiry) / (vol * math.sqrt(expiry)))
  return d_plus


def dMinusBlack(F_0, y, expiry, vol):
  '''
  Compute the d- term appearing in the Black formula
  @var F_0: forward rate at time 0
  @vary y: option strike
  @var expiry: option expiry (in years)
  @var vol: Black implied volatility
  '''
  d_minus =  (dPlusBlack(F_0 = F_0, y = y, expiry = expiry, vol = vol) - vol * math.sqrt(expiry))
  return d_minus

In [35]:
# Run Black (undiscounted) formula

F_0 = 0.02 
y = 0.03
expiry =  0.5
vol = 0.4
isCall = 1

a = Black(F_0, y, expiry, vol, isCall)
b = dPlusBlack(F_0, y, expiry, vol)
c = dMinusBlack(F_0, y, expiry, vol)

print('BOOK Black (undiscounted) price: ', round(a,10))
print('BOOK dPlusBlack: ', b)
print('BOOK dMinusBlack', c)

BOOK Black (undiscounted) price:  0.0002341801
BOOK dPlusBlack:  -1.2921142811517878
BOOK dMinusBlack -1.574956993626407


In [9]:
# First-order derivative of a function using central difference 
# Reference SABR & SABR LIBOR Market Models, page 41, table 4.6

def computeFirstDerivative(v_u_plus_du, v_u_minus_du, du):
  '''
  Compute the first derivative of a function using central difference

  @var v_u_plus_du: is the value of the function computed for a positive bump amount du
  @var v_u_minus_du: is the value of the function computed for a negative bump amount du
  @var du: bump amount
  '''
  first_derivative = (v_u_plus_du - v_u_minus_du) / (2.0 * du)

  return first_derivative

In [10]:
# First-order derivative of a function using central difference 
# Reference SABR & SABR LIBOR Market Models, page 41, table 4.6

def computeSecondDerivative(v_u, v_u_plus_du, v_u_minus_du, du):
  '''
  Compute the second derivative of a function using central difference

  @var v_u: is the value of the function
  @var v_u_plus_du: is the value of the function computed for a positive bump amount du
  @var v_u_minus_du: is the value of the function computed for a negative bump amount du
  @var du: bump amount
  '''
  second_derivative = (v_u_plus_du - 2.0*v_u + v_u_minus_du) / (du * du)

  return second_derivative

In [25]:
# Hagan et al. lognormal approximation
# Reference SABR & SABR LIBOR Market Models, page 58, table 5.1

import math
def haganLogNormalApprox(y, expiry, F_0, alpha_0, beta, nu, rho):
  '''
  Function which returns the Black implied volatility, computed using the 
  Hagan et al. lognormal approximations

  @var y: option strike
  @expiry: option expiry (in years)
  @var F_0: forward interest rate
  @var alpha_0: SABR Alpha at t=0
  @var beta: SABR Beta
  @var rho: SABR Rho
  @var nu: SABR Nu
  '''
  one_beta = 1.0 - beta
  one_betasqr =  one_beta * one_beta
  if F_0 != y:
    fK = F_0 * y
    fK_beta = math.pow(fK, one_beta / 2.0)
    log_fK = math.log(F_0 / y)
    z = nu / alpha_0 * fK_beta *log_fK
    x = math.log((math.sqrt(1.0 - 2.0 * rho * 
                            z + z * z) + z - rho) / (1 - rho))
    sigma_1 = (alpha_0 / fK_beta / (1.0 + one_betasqr / 
              24.0 * log_fK * log_fK +
              math.pow(one_beta * log_fK, 4) / 1920.0) *
              (z / x))
    sigma_exp = (one_betasqr / 24.0 * alpha_0 * alpha_0 /
                 fK_beta / fK_beta + 0.25 * rho * beta *
                 nu * alpha_0 / fK_beta + 
                 (2.0 - 3.0 * rho * rho) / 24.0 * nu * nu)
    sigma = sigma_1 * (1.0 + sigma_exp * expiry)
  else:
    f_beta = math.pow(F_0, one_beta)
    f_two_beta = math.pow(F_0, (2.0 - 2.0 * beta))
    sigma - ((alpha_0 / f_beta) * (1.0 + 
                                   ((one_betasqr / 24.0) *
                                    (alpha_0 * alpha_0 / f_two_beta) + 
                                    (0.25 * rho * beta * nu * alpha_0 / f_beta) +
                                    24.0 * nu * nu) * expiry))
  return sigma

In [26]:
# Run Hagan et al. lognormal approximation

y = 0.09
expiry = 3
F_0 = 0.03
alpha_0 = 0.2
beta = .2
nu = 0.5
rho = 0.5

x = haganLogNormalApprox(y, expiry, F_0, alpha_0, beta, nu, rho)
print('SABR Vol: ', round(x, 10))

SABR Vol:  3.272972092


In [43]:
# Run SABR calculation example

y = 0.02
expiry = 0.5
F_0 = 0.04
alpha_0 = 0.2
beta = .8
nu = 0.3
rho = -0.4

x = haganLogNormalApprox(y, expiry, F_0, alpha_0, beta, nu, rho)
round(x, 10)

0.4584267891

In [25]:
# SABR delta computation 
# Reference SABR & SABR LIBOR Market Models, page 74, table 5.2

def computeSABRDelta(y, expiry, F_0, alpha_0, beta, rho, nu, isCall):
  '''
  Compute the SABR delta

  @var y: option strike
  @var expiry: option expiry (in years)
  @var F_0: forward interest rate
  @var alpha_0: SABR Alpha at t=0
  @var beta: SABR Beta
  @var rho: SABR Rho
  @var nu: SABR Nu
  @var isCall: True or False
  '''
  small_figure = 1e-6
  
  F_0_plus_h = F_0 + small_figure
  avg_alpha_plus = (alpha_0 + (rho * nu / math.pow(F_0, beta)) * small_figure)
  vol = haganLogNormalApprox(y, expiry, F_0_plus_h, avg_alpha_plus, beta, nu, rho)
  px_f_plus_h = black(F_0_plus_h, y, expiry, vol, isCall)

  F_0_minus_h = F_0 - small_figure
  avg_alpha_minus = (alpha_0 + (rho * nu / math.pow(F_0, beta)) * (-small_figure))
  vol = haganLogNormalApprox(y, expiry, F_0_minus_h, avg_alpha_minus, beta, nu, rho)
  px_f_minus_h = black(F_0_minus_h, y, expiry, vol, isCall)
  sabr_delta = computeFirstDerivative(px_f_plus_h, px_f_minus_h, small_figure)

  return sabr_delta

In [27]:
# SABR vega computation 
# Reference SABR & SABR LIBOR Market Models, page 76, table 5.3

import math

def computeSABRVega(y, expiry, F_0, alpha_0, beta, rho, nu, isCall):
  '''
  Compute the SABR vega

  @var y: option strike
  @var expiry: option expiry (in years)
  @var F_0: forward interest rate
  @var alpha_0: SABR Alpha at t=0
  @var beta: SABR Beta
  @var rho: SABR Rho
  @var nu: SABR Nu
  @var isCall: True or False
  '''
  small_figure = 1e-6
  alpha_plus_h = alpha_0 + small_figure
  avg_F_plus = (F_0 + (rho * math.pow(F_0, beta) / nu) * small_figure)
  vol_plus = haganLogNormalApprox(y, expiry, avg_F_plus, alpha_plus_h, beta, nu, rho)
  px_a_plus_h = black(F_0, y, expiry, vol_plus, isCall)

  alpha_minus_h = alpha_0 - small_figure
  avg_F_minus = (F_0 + (rho * math.pow(F_0, beta) / nu) * (-small_figure))
  vol_minus = haganLogNormalApprox(y, expiry, avg_F_minus, alpha_minus_h, beta, nu, rho)
  px_a_minus_h = black(F_0, y, expiry, vol_minus, isCall)

  sabr_vega = computeFirstDerivative(px_a_plus_h, px_a_minus_h, small_figure)

  return sabr_vega

In [42]:
# Draw two correlated random numbers
# Reference SABR & SABR LIBOR Market Models, page 80, table 5.4

import random
# Seed the random number generator
random.seed()
import math

def drawTwoRandomNumbers(rho):
  '''
  Draw a pair of correlated random numbers
  @var rho: SABR Rho
  '''
  rand_list = []
  z1 = random.gauss(0,1)
  y1 = random.gauss(0,1)
  rand_list.append(z1)
  term1 = z1 * rho
  term2 = (y1 * math.pow((1.0 - math.pow(rho, 2.0)), 0.5))
  x2 = term1 + term2
  rand_list.append(x2)

  return rand_list

In [43]:
# Monte Carlo Euler scheme
# Reference SABR & SABR LIBOR Market Models, page 82, table 5.5

import math
def simulateSABRMonteCarloEuler(no_of_sim, no_of_steps, 
                                expiry, F_0, alpha_0, beta, rho, nu):
  '''
  Monte Carlo SABR using Euler scheme
  @var no_of_sim: Monte Carlo paths
  @var no_of_steps: discretization steps required
  to reach the option expiry date
  @var expiry: optio expiry (in years)
  @var F_0: forward interest rate
  @var alpha_0: SABR Alpha at t=0
  @var beta: SABR Beta
  @var rho: SABR Rho
  @var nu: SABR Nu
  '''
  # Step length in years
  dt = float(expiry) / float(no_of_steps)
  dt_sqrt = math.sqrt(dt)
  no_of_sim_counter = 0
  simulated_forwards = []
  while no_of_sim_counter < no_of_sim:
    F_t = F_0
    alpha_t = alpha_0
    no_of_steps_counter = 1
    while no_of_steps_counter <= no_of_steps:
      # Zero absorbing boundary used for all the beta
      # choices except beta = 0 and beta = 1
      if ((beta > 0 and beta < 1) and F_t <= 0):
        F_t = 0
        no_of_steps_counter =  no_of_steps + 1
      else:
        # Generate two correlated random numbers
        rand = drawTwoRandomNumbers(rho)
        # Simulate the forward interest rate using the Euler
        # scheme. Use the absolute for the diffusion to avoid 
        # numerical issues if the forward interest rate goes into 
        # negative territory
        dW_F = dt_sqrt * rand[0]
        F_b = math.pow(abs(F_t), beta)
        F_t = F_t + alpha_t * F_b * dW_F
        # Simulate the stochastic volatility using the Euler scheme
        dW_a = dt_sqrt * rand[1]
        alpha_t = (alpha_t + nu * alpha_t * dW_a)
      no_of_steps_counter += 1
    # At the end of each path, we store the forward interest rate in a list
    simulated_forwards.append(F_t)
    no_of_sim_counter = no_of_sim_counter + 1
  return simulated_forwards

In [48]:
# Run Monte Carlo Euler scheme example

no_of_sim = 10000
no_of_steps = 100
expiry = 1
F_0 = 0.05
alpha_0 = 0.2
beta = 0.6
rho = -0.3
nu = 0.2

see_paths_euler = simulateSABRMonteCarloEuler(no_of_sim, no_of_steps, expiry, F_0, alpha_0, beta, rho, nu)
print('See first 5 simulations: ', see_paths_euler[0:5])

See first 5 simulations:  [0.05493359128109417, 0.07716285723573395, 0.07903141866425427, 0.09761541652524656, 0.05983651717911646]


In [53]:
# Monte Carlo Milstein scheme for SABR
# Reference SABR & SABR LIBOR Market Models, page 89, table 5.10

import math
def simulateSABRMonteCarloMilstein(no_of_sim, no_of_steps, expiry, F_0, 
                                   alpha_0, beta, rho, nu):
  '''
  Monte Carlo SABR using Milstein scheme
  @var no_of_sim: Monte Carlo paths
  @var no_of_steps: discretization steps required
  to reach the option expiry date
  @var expiry: option expiry (in years)
  @var F_0: Forward interest rate
  @var alpha_0: SABR Alpha at t=0
  @var beta: SABR Beta
  @var rho: SABR Rho
  @var nu: SABR Nu
  '''
  # Step length in years
  dt = float(expiry) / float(no_of_steps)
  dt_sqrt = math.sqrt(dt)
  no_of_sim_counter = 0
  simulated_forwards = []
  while no_of_sim_counter < no_of_sim:
    F_t = F_0
    alpha_t = alpha_0
    no_of_steps_counter = 1
    while no_of_steps_counter <= no_of_steps:
      # Zero absorbing boundary used for all the beta choices
      # except beta = 0 and beta = 1
      if ((beta > 0 and beta < 1) and F_t <= 0):
        F_t = 0
        no_of_steps_counter = no_of_steps + 1
      else:
        # Generate two correlated random numbers
        rand = drawTwoRandomNumbers(rho)
        # Simulate the forward interest rate using the Milstein scheme. Use
        # the absolute for the diffusion to avoid the numerical issues if the
        # forward interest rate goes into negative territory
        dW_F = dt_sqrt * rand[0]
        F_b = math.pow(abs(F_t), beta)
        exp_F = 2.0 * beta - 1.0
        F_t = (F_t + alpha_t * F_b * dW_F 
               + 0.5 * beta * math.pow(alpha_t, 2.0) 
               * math.pow(abs(F_t), exp_F) * (rand[0] * rand[0] - 1.0) * dt )
        # Simulate the stochastic volatility using the Milstein scheme
        dW_a = dt_sqrt * rand[1]
        nu_sqr = math.pow(nu, 2.0)
        alpha_t = (alpha_t + nu * alpha_t * dW_a 
                   + 0.5 * nu_sqr * alpha_t * (rand[1] * rand[1] - 1.0) * dt)
      no_of_steps_counter += 1
    # At the end of each path, we store the forward interest rate in a list
    simulated_forwards.append(F_t)
    no_of_sim_counter = no_of_sim_counter + 1
  return simulated_forwards

In [54]:
# Run Monte Carlo Milstein example

no_of_sim = 10000
no_of_steps = 100
expiry = 1
F_0 = 0.05
alpha_0 = 0.2
beta = 0.6
rho = -0.3
nu = 0.2

see_paths_milstein = simulateSABRMonteCarloMilstein(no_of_sim, no_of_steps, expiry, F_0, alpha_0, beta, rho, nu)
print('See first 5 simulations: ', see_paths_milstein[0:5])

See first 5 simulations:  [0.02743023945507001, 0.07769470541953066, 0.049987527388471356, 0.07251551962896854, 0.01671777275693085]


In [36]:
# Double exponential correlation parameterization
# Reference SABR & SABR LIBOR Market Models, page 135, table 6.4
# Note multiple errors in code from book

import numpy as np

def generateParametricCorrelationMatrix(alpha, beta, rho_inf, maturity_grid):
  '''
  Function which generates a correlation matrix using the double 
  parameterization

  @var alpha: alpha parameter
  @var beta: beta parameter
  @var rho_inf: rho_inf parameter
  @var maturity_grid: represents a list containing the firward rate maturities
  which we are going to model. For example, if we want to model the
  semi-annual forward rates maturing between 1 and 5 years from now, we will set:
  maturity_grid = [1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5]
  '''
  grid = len(maturity_grid)
  # Create a null correlation matrix
  corr_matrix = np.zeros((grid, grid))
  for i in range(grid):
    for j in range(grid):
      first_e = np.exp(-beta * abs(maturity_grid[i] 
                                   - maturity_grid[j]))
      second_e = np.exp(-alpha * min(maturity_grid[i], 
                                     maturity_grid[j]))
      corr_matrix[i,j] = (rho_inf + (1.0 - rho_inf) *
                          first_e * second_e) 
  return corr_matrix

In [37]:
# Generate parametric correlation matrix example

alpha = 0.2
beta = 0.6
rho_inf = -0.3
maturity_grid = [1, 2, 3, 4, 5]

corr = generateParametricCorrelationMatrix(alpha, beta, rho_inf, maturity_grid)
corr

array([[ 0.76434998,  0.28412765,  0.02057605, -0.12406413, -0.20344435],
       [ 0.28412765,  0.57141606,  0.17824327, -0.03753453, -0.15595589],
       [ 0.02057605,  0.17824327,  0.41345513,  0.09155248, -0.08511145],
       [-0.12406413, -0.03753453,  0.09155248,  0.28412765,  0.02057605],
       [-0.20344435, -0.15595589, -0.08511145,  0.02057605,  0.17824327]])

In [56]:
# Compute the Black volatility in terms of the instantaneous volatility
# parameterized using equation 6.26 on page 138
# Reference SABR & SABR LIBOR Market Models, page 139, table 6.6

import numpy as np

def getCapletVolatility(expiry, a, b, c, d):
  '''
  Return the caplet volatility at time t=0, computed in terms of the 
  instantaneous volatility function form proposed by Rebonato

  @var expiry: caplet expiry (in years)
  @var a: parameter a of Rebonato's instant. vol function
  @var b: parameter b of Rebonato's instant. vol function
  @var c: parameter c of Rebonato's instant. vol function
  @var d: parameter d of Rebonato's instant. vol function
  '''
  a_sqr = a * a
  b_sqr = b * b
  c_sqr = c * c
  d_sqr = d * d
  tau = expiry
  exp_term = np.exp(-c *  tau)
  exp_plus_term = np.exp(c *  tau)
  term1 = (-2.0 * c_sqr * (a_sqr + 4.0 * a * d * exp_plus_term 
                           - 2.0 * c * d_sqr * tau * exp_plus_term * exp_plus_term))
  term2 = (2.0 * b * c * (2.0 * a * c * tau + a 
                          + 4.0 * d * exp_plus_term * (c * tau + 1.0)))
  term3 = (b_sqr * (-(2.0 * c_sqr * tau * tau + 2.0 * c * tau + 1.0)))
  variance_expiry = ((1.0 / (4.0 * c * c * c)) * exp_term * exp_term * (term1 - term2 + term3))
  
  tau = 0
  exp_term = np.exp(-c * tau)
  exp_plus_term = np.exp(c * tau)
  term1 = (-2.0 * c_sqr * (a_sqr + 4.0 * a * d * exp_plus_term 
                           - 2.0 * c * d_sqr * tau * exp_plus_term * exp_plus_term))
  term2 = (2.0 * b * c * (2.0 * a * c * tau + a 
                          + 4.0 * d * exp_plus_term * (c * tau + 1.0)))
  term3 = (b_sqr * (-(2.0 * c_sqr * tau * tau + 2.0 * c * tau + 1.0)))
  variance_t0 = ((1.0 / (4.0 * c * c * c)) * exp_term * exp_term * (term1 - term2 + term3))
  
  variance = variance_expiry - variance_t0
  volatility = np.sqrt(variance / expiry)
  
  return volatility

In [57]:
# Compute the instantaneous volatility in terms of the parametric form
# Reference SABR & SABR LIBOR Market Models, page 141, table 6.7

import numpy as np

def getInstantaneousVolatility(t, expiry, a, b, c, d):
  '''
  Return the instantaneous volatility computed in terms of the parametric
  form proposed by Rebonato at a given time t

  @var t: time at which we want to compute the instantaneous volatility (in years)
  @var expiry: caplet expiry (in years)
  @var a: parameter a of Rebonato's instant. vol function
  @var b: parameter b of Rebonato's instant. vol function
  @var c: parameter c of Rebonato's instant. vol function
  @var d: parameter d of Rebonato's instant. vol function
  '''
  tau = expiry - t
  instantaneous_vol = (a + b * tau) * np.exp(-c * tau) + d

  return instantaneous_vol